<a href="https://colab.research.google.com/github/Madhu-2101/LangChainAutoReply/blob/main/Email_Handler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installing required libraries

!apt-get install sqlite3
!pip -q install langchain
!pip -q install langchain-groq duckduckgo-search
!pip -q install -U langchain_community tiktoken langchainhub
!pip -q install -U langchain langgraph tavily-python
!pip show langgraph
!pip -q install faiss-cpu
!pip install ipywidgets
!pip -q install langchain_google_genai

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
sqlite3 is already the newest version (3.37.2-2ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Name: langgraph
Version: 0.1.15
Summary: Building stateful, multi-actor applications with LLMs
Home-page: https://www.github.com/langchain-ai/langgraph
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: langchain-core
Required-by: 


In [ ]:
import sqlite3
import os
import ipywidgets as widgets
from google.colab import userdata
from pprint import pprint
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.document_loaders import TextLoader
from IPython.display import display, clear_output

In [ ]:

def setup_equipment_table():
    conn = sqlite3.connect('film_equipment_rental.db')
    cur = conn.cursor()

    # Create Equipment table
    cur.execute('''
    CREATE TABLE IF NOT EXISTS Equipments (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        category TEXT NOT NULL,
        availability TEXT NOT NULL,
        price REAL
    )
    ''')

    # Insert sample data
    cur.execute("INSERT INTO Equipments (name, category, availability, price) VALUES (?, ?, ?, ?)", ('sony', 'camera', 'true', 599.99))
    cur.execute("INSERT INTO Equipments (name, category, availability, price) VALUES (?, ?, ?, ?)", ('dslr', 'camera', 'false', 399.99))
    cur.execute("INSERT INTO Equipments (name, category, availability, price) VALUES (?, ?, ?, ?)", ('canon', 'camera', 'true', 799.99))
    cur.execute("INSERT INTO Equipments (name, category, availability, price) VALUES (?, ?, ?, ?)", ('nikon', 'camera', 'true', 899.99))
    conn.commit()
    cur.close()
    conn.close()

setup_equipment_table()


In [ ]:
def verify_new_database():
    conn = sqlite3.connect('film_equipment_rental.db')
    cur = conn.cursor()

    # Query to select all data from Equipment table
    cur.execute("SELECT * FROM Equipments")
    rows = cur.fetchall()

    for row in rows:
        print(row)

    cur.close()
    conn.close()

verify_new_database()


(1, 'sony', 'camera', 'true', 599.99)
(2, 'dslr', 'camera', 'false', 399.99)
(3, 'canon', 'camera', 'true', 799.99)
(4, 'nikon', 'camera', 'true', 899.99)


In [ ]:
def get_product_info(product_name):
    """
    Retrieves the price of the product if available,
    or provides similar items if the product is not available.

    Args:
    - product_name (str): The name of the product to check.

    Returns:
    - dict: A dictionary containing the product price if available,
            or a list of similar items if not available.
    """
    conn = sqlite3.connect('film_equipment_rental.db')
    cur = conn.cursor()

    query = """
        SELECT name, price
        FROM Equipments
        WHERE name = ? AND availability = 'true'

        UNION ALL

        SELECT name, price
        FROM Equipments
        WHERE category = (
            SELECT category
            FROM Equipments
            WHERE name = ?
        )
        AND name <> ?
        AND availability = 'true'
        LIMIT 3;
    """

    cur.execute(query, (product_name, product_name, product_name))
    results = cur.fetchall()

    cur.close()
    conn.close()

    # Check if the product is available
    if any(result[0] == product_name for result in results):
        # Return price of the specific product if found
        price = next(result[1] for result in results if result[0] == product_name)
        return {
            "name": product_name,
            "available": True,
            "price": price
        }
    else:
        # Return similar items if the product is not available
        similar_items = [{"name": row[0], "price": row[1]} for row in results]
        return {
            "name": product_name,
            "available": False,
            "similar_items": similar_items
        }

In [ ]:

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('default')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
GROQ_LLM = ChatGroq(
            model="llama3-70b-8192",
        )

In [ ]:
prompt = PromptTemplate(
    template="""system
    You are an expert Email Categorizer Agent. Your job is to accurately categorize an email into one of the following categories based on its content:

    - negative_review_handling: for complaints or negative feedback.
    - product_enquiry: for inquiries about film equipments pricing only.
    - positive_review_handling: for positive feedback or praise.
    - assistance_request_handling: for requests for help or support with a product or any enqury about the film equipments.
    - general_handling: for emails that do not fit into any of the above categories.

    Provide only one category from the list above.

    EMAIL CONTENT:\n\n {initial_email} \n\n

    assistant
    """,
    input_variables=["initial_email"],
)

email_category_generator = prompt | GROQ_LLM | StrOutputParser()

In [ ]:
# Define the prompt template for extracting the product name

search_product_name_prompt = PromptTemplate(
    template="""system
    You are skilled at finding just the product name from an email (if the email contains 'sony camera', the product name should be considered as only 'sony') to search them in the database for their price.

    Based on the INITIAL_EMAIL find the product name.
    The parsed product name should be one word.
    Just return the product name only.

    INITIAL_EMAIL: {initial_email} \n
    assistant""",
    input_variables=["initial_email"],
)

# Generate product name
product_name_generator = search_product_name_prompt | GROQ_LLM | StrOutputParser()




# Define the prompt template for crafting the product review response
product_review_prompt = PromptTemplate(
    template="""system
    You are skilled at crafting responses to product inquiries based on INITIAL_EMAIL AND PRODUCT_DETAILS.
    Your task is to handle an email inquiry about a product by providing a reply that includes the price of the product if it is available from the given PRODUCT_DETAILS (e.g., PRODUCT_DETAILS 'name': 'sony', 'available': True, 'price': 599.99) if the product is available, just reply with the price.

    If the product is not available (e.g., PRODUCT_DETAILS 'name': 'dslr', 'available': False, 'similar_items': ['name': 'sony', 'price': 599.99, 'name': 'canon', 'price': 799.99, 'name': 'sony', 'price': 599.99]), you should suggest similar products from the product_details.
    PRODUCT_DETAILS is the data extracted from the database. The generated response should be related to PRODUCT_DETAILS ONLY.
    Based on the INITIAL_EMAIL and PRODUCT_DETAILS, generate the response.
    Always sign off the email in an appropriate manner from XXXX, Customer Service.
    Return the draft email as JSON with a single key 'draft_email' and no premable or explaination.

    INITIAL_EMAIL: {initial_email} \n
    PRODUCT_DETAILS: {product_details} \n
    assistant""",
    input_variables=["initial_email", "product_details"],
)



# Generate the product review response

product_review_generator = product_review_prompt | GROQ_LLM | StrOutputParser()



In [ ]:
# Define the prompt template for handling reviews

draft_review_generating_prompt = PromptTemplate(
    template="""system
    You are the Email Writer Agent. Use the INITIAL_EMAIL and EMAIL_CATEGORY to write a thoughtful and helpful response.

    - For 'negative_review_handling': Escalate to the CRM system for follow-up with a phone call from customer service and offer a gift voucher in the reply.
    - For 'positive_review_handling': Thank the sender and encourage them to share their experience on social media.
    - For 'general_handling': give a message stating that the email will forwarded to customer service for further evaluation with a thoughtful response
    Always sign off the email in an appropriate manner from XXXX, Customer Service.

    Return the draft email as JSON with a single key 'draft_email' and no premable or explaination.

    INITIAL_EMAIL: {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    assistant""",
    input_variables=["initial_email", "email_category"],
)

# Generate the review handling response

draft_review_generator = draft_review_generating_prompt | GROQ_LLM | StrOutputParser()


In [ ]:
prompt=ChatPromptTemplate.from_template(
"""
Answer the questions based on the provided context only.
Please provide the most accurate response based on the question
<context>
{context}
<context>
Questions:{input}

"""
)

In [ ]:
def vector_embedding():
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    loader = TextLoader("/content/sample_data/FAQs.txt")  # Data Ingestion
    docs = loader.load()  # Document Loading
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)  # Chunk Creation
    final_documents = text_splitter.split_documents(docs[:20])  # Splitting
    vectors = FAISS.from_documents(final_documents, embeddings)  # Vector OpenAI embeddings
    return vectors



In [ ]:

def process_question(question):
    vectors = vector_embedding()
    document_chain = create_stuff_documents_chain(GROQ_LLM, prompt)
    retriever = vectors.as_retriever()
    retrieval_chain = create_retrieval_chain(retriever, document_chain)

    response = retrieval_chain.invoke({'input': question})

    return response['answer']


In [ ]:

# Define the prompt template for handling reviews

review_generating_prompt = PromptTemplate(
    template="""system
    You are the draft Email Writer Agent. Use the INITIAL_EMAIL and EMAIL_CATEGORY and FAQ_answer to write a thoughtful assistance request response.
    FAQ_answer contains the answer generated by a RAG pipeline. Your response should be based on and should be related FAQ_answer ONLY.

    You response should be related to the objective of the FAQ_answer only.
    If there is no proper answer for the INITIAL_EMAIL in given FAQ_answer then give a response stating that the issue will be escalated to customer service
    Always sign off the email in an appropriate manner from XXXX, Customer Service.

    Return the draft email as JSON with a single key 'draft_email' and no premable or explaination.

    INITIAL_EMAIL: {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    FAQ_answer: {FAQ_answer} \n
    assistant""",
    input_variables=["initial_email", "email_category", "FAQ_answer"],
)


# Generate the review handling response

review_generator = review_generating_prompt | GROQ_LLM | StrOutputParser()


In [ ]:
# Rewrite Email with Analysis

final_email_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the Final Email Agent read the INITIAL_EMAIL and EMAIL_CATEGORY and DRAFT_EMAIL
    and use it to just rewrite the DRAFT_EMAIL to create a simple final email without removing or adding any other information that hasn't been given by the INITIAL_EMAIL and DRAFT_EMAIL..



    Return the final response as JSON with a single key 'final_email' and no premable or explaination.

    Always sign off the email in an appropriate manner from Jhon, Customer Service.

    Only return the final email content
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n\n
    EMAIL_CATEGORY: {email_category} \n\n
    DRAFT_EMAIL: {draft_email} \n\n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|""",
    input_variables=["initial_email",
                     "email_category",
                     "draft_email",
                     ],
)

final_review_generator = final_email_prompt | GROQ_LLM | JsonOutputParser()


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Define the text input and output widgets
email_input = widgets.Textarea(
    value='',
    placeholder='Enter the email content here...',
    description='Email:',
    layout=widgets.Layout(width='100%', height='200px')
)

response_output = widgets.Output()

def process_email(button):
    # Clear previous output
    with response_output:
        clear_output()

        email_content = email_input.value
        if not email_content:
            print("Please enter an email content.")
            return

        # Categorize the email
        email_category = email_category_generator.invoke({"initial_email": email_content})

        # Check category and generate response
        if email_category == 'product_enquiry':
            # Extract product name
            product_name = product_name_generator.invoke({"initial_email": email_content})
            product_details = get_product_info(product_name)
            formatted_product_details = {
                'name': product_details['name'],
                'available': product_details['available'],
                'price': product_details.get('price'),
                'similar_items': product_details.get('similar_items', [])
            }

            # Generate product review response
            draft_email = product_review_generator.invoke({
                "initial_email": email_content,
                "product_details": formatted_product_details
            })

        elif email_category in ['positive_review_handling', 'negative_review_handling', 'general_handling']:
            draft_email = draft_review_generator.invoke({
                "initial_email": email_content,
                "email_category": email_category
            })

        elif email_category == 'assistance_request_handling':
            FAQ_answer = process_question(email_content)
            draft_email = review_generator.invoke({
                "initial_email": email_content,
                "email_category": email_category,
                "FAQ_answer": FAQ_answer
            })

        # Finalize email
        final_email = final_review_generator.invoke({
            "initial_email": email_content,
            "email_category": email_category,
            "draft_email": draft_email
        })


        print(f"\n{final_email.get('final_email', 'No response generated')}")

# Define the button widget
process_button = widgets.Button(
    description='Get Response',
    button_style='success'
)

# Link the button to the callback function
process_button.on_click(process_email)

# Display the UI
display(email_input, process_button, response_output)


Textarea(value='', description='Email:', layout=Layout(height='200px', width='100%'), placeholder='Enter the e…

Button(button_style='success', description='Get Response', style=ButtonStyle())

Output()